Data Loading and Model Architecture Configuration

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, datasets
from torchvision.utils import save_image
import os
from PIL import Image
import numpy as np
import json



device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
batch_size = 64
image_size = 256
nz = 100  
nc = 3    
ngf = 64  
ndf = 64  
num_epochs = 100
lr = 0.0002
beta1 = 0.5
embedding_dim = 100  # Dimension of text embeddings

# Create a dataset
class FloorplanDataset(Dataset):
    def __init__(self, image_dir, tags_dir, transform=None):
        self.image_dir = image_dir
        self.tags_dir = tags_dir
        self.transform = transform
        self.filenames = [f for f in os.listdir(image_dir) if os.path.isfile(os.path.join(tags_dir, f.replace('png', 'txt')))]

    def __len__(self):
        return len(self.filenames)

    def __getitem__(self, idx):
        img_name = os.path.join(self.image_dir, self.filenames[idx])
        image = Image.open(img_name).convert('RGB')
        
        tag_name = os.path.join(self.tags_dir, self.filenames[idx].replace('png', 'txt'))
        with open(tag_name, 'r',encoding = "utf8") as file:
            tags = file.read().replace('\n', '')
        
        if self.transform:
            image = self.transform(image)
        
        
        tag_embedding = torch.zeros(embedding_dim)
        words = tags.split()
        for word in words:
            index = abs(hash(word)) % embedding_dim
            tag_embedding[index] = 1

        return image, tag_embedding

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

dataset = FloorplanDataset(image_dir='Dataset/input/floorplan/floorplan_image', tags_dir='Dataset/input/tags/human_annotated_tags', transform=transform)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

Conditional GAN Model Definition

In [2]:
class Generator(nn.Module):
    def __init__(self, nz, ngf, nc, embedding_dim):
        super(Generator, self).__init__()
        self.text = nn.Linear(embedding_dim, nz) 
        self.main = nn.Sequential(
            
            nn.ConvTranspose2d(nz * 2, ngf * 16, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 16),
            nn.ReLU(True),
            nn.ConvTranspose2d(ngf * 16, ngf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            nn.ConvTranspose2d(ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            nn.ConvTranspose2d(ngf * 2, ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            nn.ConvTranspose2d(ngf, ngf // 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf // 2),
            nn.ReLU(True),
            nn.ConvTranspose2d(ngf // 2, nc, 4, 2, 1, bias=False),
            nn.Tanh()
        )

    def forward(self, noise, text_embedding):
        
        text_embedding = self.text(text_embedding)
        text_embedding = text_embedding.view(noise.size(0), nz, 1, 1)  

        combined_input = torch.cat([noise, text_embedding], 1)
        return self.main(combined_input)

class Discriminator(nn.Module):
    def __init__(self, ndf, nc, image_size, embedding_dim):
        super(Discriminator, self).__init__()
        self.text = nn.Linear(embedding_dim, image_size * image_size)  
        self.main = nn.Sequential(
            nn.Conv2d(nc + 1, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf * 8, ndf * 16, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 16),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf * 16, ndf * 32, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 32),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf * 32, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()  
        )

    def forward(self, image, text_embedding):
        
        text_embedding = self.text(text_embedding).view(-1, 1, image_size, image_size)
        
        input = torch.cat([image, text_embedding], 1)
        return self.main(input).view(-1)  

Evaluation Metrics and Computation Functions

In [3]:
import torch
import torch.nn.functional as F
from torchvision.models import inception_v3
import numpy as np
from scipy.linalg import sqrtm

# Initialize Inception model
inception_model = inception_v3(pretrained=True, transform_input=False).to(device)
inception_model.eval()

def calculate_fid(real_imgs, fake_imgs, epsilon=1e-6):
    # Ensure the Inception model is in eval mode and gradients are off
    inception_model.eval()
    with torch.no_grad():
        # Get features from the Inception model, ensure they are detached from the graph and moved to CPU
        real_activations = inception_model(real_imgs).detach().cpu().numpy()
        fake_activations = inception_model(fake_imgs).detach().cpu().numpy()

    # Calculate mean and covariance statistics for real and fake activations
    mu1, sigma1 = real_activations.mean(axis=0), np.cov(real_activations, rowvar=False)
    mu2, sigma2 = fake_activations.mean(axis=0), np.cov(fake_activations, rowvar=False)

    # Adding a small epsilon to the diagonal of covariance matrices to ensure they are positive semi-definite
    sigma1 += np.eye(sigma1.shape[0]) * epsilon
    sigma2 += np.eye(sigma2.shape[0]) * epsilon

    # Calculate the sum of squared differences between the means
    ssdiff = np.sum((mu1 - mu2) ** 2.0)

    # Calculate sqrt of product of covariance matrices
    covmean = sqrtm(sigma1.dot(sigma2))

    # Check for complex numbers resulting from numerical inaccuracies and take the real part
    if np.iscomplexobj(covmean):
        covmean = covmean.real

    # Calculate the Frechet Inception Distance
    fid = ssdiff + np.trace(sigma1 + sigma2 - 2.0 * covmean)
    return fid

def calculate_inception_score(imgs, n_split=10):
    # Calculate probabilities using inception model
    with torch.no_grad():
        preds = F.softmax(inception_model(imgs), dim=1).detach().cpu().numpy()

    # Split predictions and calculate scores
    scores = []
    n_part = preds.shape[0] // n_split
    for i in range(n_split):
        part = preds[i * n_part:(i + 1) * n_part, :]
        kl_div = part * (np.log(part) - np.log(np.expand_dims(part.mean(axis=0), 0)))
        kl_div = kl_div.sum(axis=1)
        scores.append(np.exp(kl_div.mean()))
    return np.mean(scores)

D:\LandTech\venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
D:\LandTech\venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Model Training

In [4]:
# Create the generator and discriminator
netG = Generator(nz, ngf, nc, embedding_dim).to(device)
netD = Discriminator(ndf, nc, image_size, embedding_dim).to(device)

# Setup Adam optimizers for both G and D
optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(beta1, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(beta1, 0.999))

# Loss function
criterion = nn.BCELoss()

# Noise for input to the generator
fixed_noise = torch.randn(batch_size, nz, 1, 1, device=device)
real_label = 1
fake_label = 0
G_losses = []
D_losses = []
real_acc_list = []
fake_acc_list = []
FID_scores = []
IS_scores = []

# Training Loop
for epoch in range(num_epochs):
    real_correct = 0
    fake_correct = 0
    total_real = 0
    total_fake = 0

    for i, data in enumerate(dataloader, 0):
        # Update D network: maximize log(D(x)) + log(1 - D(G(z)))
        ## Train with all-real batch
        netD.zero_grad()
        real_cpu = data[0].to(device)
        batch_size = real_cpu.size(0)
        
        # Apply label smoothing here for the real labels
        real_labels = torch.full((batch_size,), 0.9, dtype=torch.float, device=device)  # Soft labels for real
        fake_labels = torch.full((batch_size,), 0.1, dtype=torch.float, device=device)  # Soft labels for fake

        output = netD(real_cpu, data[1].to(device))
        errD_real = criterion(output, real_labels)
        errD_real.backward()
        D_x = output.mean().item()

        # Calculate real accuracy
        pred_real = output.detach() > 0.5  # True if discriminator thinks image is real
        real_correct += pred_real.sum().item()
        total_real += batch_size

        ## Train with all-fake batch
        noise = torch.randn(batch_size, nz, 1, 1, device=device)
        fake = netG(noise, data[1].to(device))
        output = netD(fake.detach(), data[1].to(device))
        errD_fake = criterion(output, fake_labels)
        errD_fake.backward()
        D_G_z1 = output.mean().item()

        # Calculate fake accuracy
        pred_fake = output.detach() < 0.5  # True if discriminator thinks image is fake
        fake_correct += pred_fake.sum().item()
        total_fake += batch_size

        errD = errD_real + errD_fake
        optimizerD.step()

        D_losses.append(errD.item())

        # Update G network: maximize log(D(G(z)))
        netG.zero_grad()
        output = netD(fake, data[1].to(device))
        errG = criterion(output, real_labels)  # Fake labels are real for generator cost
        errG.backward()
        D_G_z2 = output.mean().item()
        optimizerG.step()

        G_losses.append(errG.item())
        
    # Metric calculation every 5 epochs
    if (epoch + 1) % 5 == 0:
        with torch.no_grad():
            # Fetch a batch and separate images and embeddings
            data = next(iter(dataloader))
            real_images, embeddings = data[0].to(device), data[1].to(device)

            # Generate fake images for evaluation
            fake_images = netG(fixed_noise, embeddings).to(device)  # Use the same embeddings for real and fake images

            # Ensure the images are suitable for Inception V3
            real_images_resized = F.interpolate(real_images, size=(299, 299), mode='bilinear', align_corners=False)
            fake_images_resized = F.interpolate(fake_images, size=(299, 299), mode='bilinear', align_corners=False)

            # Calculate FID and IS
            fid = calculate_fid(real_images_resized, fake_images_resized)
            inception_score = calculate_inception_score(fake_images_resized)

            FID_scores.append(fid)
            IS_scores.append(inception_score)
            print(f'Epoch {epoch+1}: FID = {fid}, IS = {inception_score}')


        # Output training stats
        if i % 50 == 0:
            print('[%d/%d][%d/%d]\tLoss_D: %.4f\tLoss_G: %.4f\tD(x): %.4f\tD(G(z)): %.4f / %.4f'
                  % (epoch, num_epochs, i, len(dataloader),
                     errD.item(), errG.item(), D_x, D_G_z1, D_G_z2))

    # Calculate accuracies for the entire epoch
    real_accuracy = real_correct / total_real
    fake_accuracy = fake_correct / total_fake
    real_acc_list.append(real_accuracy)
    fake_acc_list.append(fake_accuracy)
    print(f'Epoch {epoch + 1} Real Accuracy: {real_accuracy:.4f}, Fake Accuracy: {fake_accuracy:.4f}')

    # Check how the generator is doing by saving G's output on fixed_noise
    if (epoch == 0) or ((epoch + 1) % 10 == 0):
        with torch.no_grad():
            fake = netG(torch.randn(35, nz, 1, 1, device=device), torch.randn(batch_size, embedding_dim, device=device)).detach().to(device)
        output_dir = '/Dataset/output'
        os.makedirs(output_dir, exist_ok=True)
        save_image(fake, f'/Dataset/output/fake_samples_epoch_{epoch}.png', normalize=True)

# Save the final trained model
torch.save(netG.state_dict(), 'generator.pth')
torch.save(netD.state_dict(), 'discriminator.pth')

Evaluation Graphs

In [5]:
# import matplotlib.pyplot as plt

# epochs = list(range(1, num_epochs + 1))
# metric_epochs = list(range(5, num_epochs + 1, 5))
# iterations = list(range(1, len(D_losses) + 1))

# # Plotting Discriminator and Generator Losses
# plt.figure(figsize=(10, 5))
# plt.plot(iterations, D_losses, label='Discriminator Loss')
# plt.plot(iterations, G_losses, label='Generator Loss')
# plt.title('Training Losses')
# plt.xlabel('Epochs')
# plt.ylabel('Loss')
# plt.legend()
# plt.grid(True)
# plt.show()

# # Plotting Real and Fake Accuracy
# plt.figure(figsize=(10, 5))
# plt.plot(epochs, real_acc_list, label='Real Accuracy')
# plt.plot(epochs, fake_acc_list, label='Fake Accuracy')
# plt.title('Discriminator Accuracy')
# plt.xlabel('Epochs')
# plt.ylabel('Accuracy')
# plt.legend()
# plt.grid(True)
# plt.show()

# # Plotting FID and IS Scores
# plt.figure(figsize=(10, 5))
# plt.plot(metric_epochs, FID_scores, label='FID')
# plt.title('Frechet Inception Distance')
# plt.xlabel('Epochs')
# plt.ylabel('FID')
# plt.legend()
# plt.grid(True)
# plt.show()

# plt.figure(figsize=(10, 5))
# plt.plot(metric_epochs, IS_scores, label='Inception Score')
# plt.title('Inception Scores')
# plt.xlabel('Epochs')
# plt.ylabel('Score')
# plt.legend()
# plt.grid(True)
# plt.show()

Generate Image

In [6]:
import torchvision.transforms as transforms
from torchvision.utils import save_image
from PIL import Image
import os
from IPython.display import display  # For displaying images in Jupyter notebooks

# Load pre-trained Generator
netG = Generator(nz, ngf, nc, embedding_dim).to(device)
netG.load_state_dict(torch.load('generator.pth', map_location=device))
netG.eval()

# Function to generate an image from text
def generate_image_from_text(text):
    # Convert text to embedding (simplified one-hot encoding as an example)
    tag_embedding = torch.zeros(embedding_dim)
    words = text.split()
    for word in words:
        index = abs(hash(word)) % embedding_dim
        tag_embedding[index] = 1

    # Generate noise vector
    noise = torch.randn(1, nz, 1, 1, device=device)
    tag_embedding = tag_embedding.to(device).unsqueeze(0)  # Add batch dimension

    # Generate image
    with torch.no_grad():
        fake_image = netG(noise, tag_embedding)
    
    # Convert image tensor to PIL Image
    fake_image = (fake_image * 0.5 + 0.5)  # unnormalize
    img = transforms.ToPILImage()(fake_image.squeeze(0).cpu())
    img.save(f'generated_image.png')  # Save the image

    # Display the image
    return img

# generate_image_from_text("Balcony 1 is at the South corner position. between bathroom and kitchen. it is 8x8 sq ft Balcony 2 is at the North middle of the corner position. near living room. it is 14x3 sq ft. The bathroom is at the bathroom is center position in between common room2 and living room. it is 8x5 sq ft. Common room 1 is at the North middle of the corner position. between master room and living room. it is 10x10 sq ft. Common room 2 is at the West corner position. between bathroom and living room. it is 10x8 sq ft. The kitchen is at the South middle corner position. between living room and balcony. it is 10x5 sq ft. Living room is east center position it is south face. near all room located. it is  10x20 sq ft. The master room is at the West corner position. between common room1 and common room2. it is 10x10 sq ft.  ")

C:\Users\DEEPAK\AppData\Local\Temp\ipykernel_16480\319299788.py:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  netG.load_state_dict(torch.load('generator.pth', map_locatio